In [1]:
import streamlit as st
import pandas as pd
import numpy as np
import mysql.connector
from datetime import datetime
import matplotlib.pyplot as plt
import seaborn as sns

# Configuration - Update these values
config = {
    'host': 'localhost',
    'user': 'root',
    'password': '',
    'database': 'data_warehouse'
}

# Initialize MySQL database connection
def get_db_connection():
    try:
        conn = mysql.connector.connect(**config)
        return conn
    except mysql.connector.Error as err:
        st.error(f"Database connection error: {err}")
        return None

# Initialize database tables
def init_db():
    conn = get_db_connection()
    if conn:
        try:
            cursor = conn.cursor()
            cursor.execute('''CREATE TABLE IF NOT EXISTS predictions (
                             id INT AUTO_INCREMENT PRIMARY KEY,
                             timestamp DATETIME,
                             attendance FLOAT,
                             assignment_score FLOAT,
                             midterm_score FLOAT,
                             final_score FLOAT,
                             outstanding_balance FLOAT,
                             prediction FLOAT,
                             INDEX (timestamp))''')
            conn.commit()
        except mysql.connector.Error as err:
            st.error(f"Database initialization error: {err}")
        finally:
            conn.close()

# Save prediction to database
def save_prediction(attendance, assignment, midterm, final, balance, prediction):
    conn = get_db_connection()
    if conn:
        try:
            cursor = conn.cursor()
            timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
            cursor.execute('''INSERT INTO predictions 
                             (timestamp, attendance, assignment_score, midterm_score, 
                              final_score, outstanding_balance, prediction)
                             VALUES (%s, %s, %s, %s, %s, %s, %s)''',
                          (timestamp, attendance, assignment, midterm, final, balance, prediction))
            conn.commit()
            st.success("Prediction saved to database successfully!")
        except mysql.connector.Error as err:
            st.error(f"Failed to save prediction: {err}")
        finally:
            conn.close()

# Get prediction history
def get_prediction_history(limit=100):
    conn = get_db_connection()
    if conn:
        try:
            query = f'''SELECT * FROM predictions 
                        ORDER BY timestamp DESC 
                        LIMIT {limit}'''
            df = pd.read_sql(query, conn)
            return df
        except mysql.connector.Error as err:
            st.error(f"Failed to fetch prediction history: {err}")
            return pd.DataFrame()
        finally:
            conn.close()
    return pd.DataFrame()

# Initialize database
init_db()

# Dashboard title
st.title("📊 Student Performance Prediction Dashboard")

# Sidebar for user inputs
with st.sidebar:
    st.header("⚙️ Model Input Parameters")
    
    attendance = st.slider("Attendance (%)", 0, 100, 75)
    assignment_score = st.slider("Assignment Score", 0, 100, 70)
    midterm_score = st.slider("Midterm Score", 0, 100, 65)
    final_score = st.slider("Final Score", 0, 100, 60)
    outstanding_balance = st.number_input("Outstanding Balance ($)", 0, 10000, 500)
    
    # Mock prediction function (replace with actual model prediction)
    def predict_performance(attendance, assignment, midterm, final, balance):
        # This is a placeholder - replace with your actual model prediction
        weights = np.array([0.2, 0.25, 0.25, 0.25, -0.0001])
        inputs = np.array([attendance, assignment, midterm, final, balance])
        raw_score = np.dot(weights, inputs)
        probability = 1 / (1 + np.exp(-0.1 * (raw_score - 50)))  # Sigmoid function
        return probability
    
    if st.button("Predict Performance"):
        prediction = predict_performance(attendance, assignment_score, midterm_score, 
                                       final_score, outstanding_balance)
        save_prediction(attendance, assignment_score, midterm_score, 
                       final_score, outstanding_balance, prediction)
        st.session_state['prediction'] = prediction
        st.session_state['input_data'] = {
            'Attendance': attendance,
            'Assignment_Score': assignment_score,
            'Midterm_Score': midterm_score,
            'Final_Score': final_score,
            'Outstanding_Balance': outstanding_balance
        }

# Main dashboard layout
col1, col2 = st.columns([2, 1])

with col1:
    st.subheader("📈 Feature Analysis")
    
    # Feature importance visualization (mock data)
    feature_importance = pd.DataFrame({
        'Feature': ['Attendance', 'Assignment_Score', 'Midterm_Score', 'Final_Score', 'Outstanding_Balance'],
        'Importance': [0.20, 0.25, 0.25, 0.25, -0.05]  # Mock importance values
    })
    
    fig, ax = plt.subplots(figsize=(10, 5))
    sns.barplot(data=feature_importance, x='Importance', y='Feature', palette='viridis')
    plt.title("Feature Importance in Model")
    st.pyplot(fig)
    
    # Correlation heatmap (mock data)
    st.subheader("🔍 Feature Correlations")
    mock_data = pd.DataFrame({
        'Attendance': np.random.normal(attendance, 10, 100),
        'Assignment_Score': np.random.normal(assignment_score, 10, 100),
        'Midterm_Score': np.random.normal(midterm_score, 10, 100),
        'Final_Score': np.random.normal(final_score, 10, 100),
        'Outstanding_Balance': np.random.normal(outstanding_balance, 200, 100)
    })
    
    fig2, ax2 = plt.subplots(figsize=(8, 6))
    sns.heatmap(mock_data.corr(), annot=True, cmap='coolwarm', ax=ax2)
    st.pyplot(fig2)

with col2:
    st.subheader("🔮 Performance Prediction")
    
    if 'prediction' in st.session_state:
        # Display prediction result
        prediction = st.session_state['prediction']
        st.metric("Success Probability", f"{prediction*100:.1f}%")
        
        # Visual gauge
        fig3, ax3 = plt.subplots(figsize=(6, 2))
        ax3.barh(['Success'], [prediction*100], color='#4CAF50')
        ax3.set_xlim(0, 100)
        ax3.set_title("Performance Probability")
        st.pyplot(fig3)
        
        # Display input values
        st.subheader("📝 Input Summary")
        input_data = st.session_state['input_data']
        for key, value in input_data.items():
            st.write(f"{key}: {value}")
    else:
        st.info("Adjust the parameters in the sidebar and click 'Predict Performance'")

# Prediction history section
st.subheader("📋 Prediction History")
history_df = get_prediction_history()

if not history_df.empty:
    # Show latest predictions
    st.dataframe(history_df.head(10))
    
    # Show prediction trend over time
    history_df['timestamp'] = pd.to_datetime(history_df['timestamp'])
    history_df.set_index('timestamp', inplace=True)
    
    fig4, ax4 = plt.subplots(figsize=(10, 4))
    history_df['prediction'].plot(ax=ax4, marker='o')
    plt.title("Prediction Trend Over Time")
    plt.ylabel("Success Probability")
    plt.grid(True)
    st.pyplot(fig4)
    
    # Show statistics
    st.subheader("📊 Prediction Statistics")
    col1, col2, col3 = st.columns(3)
    with col1:
        st.metric("Total Predictions", len(history_df))
    with col2:
        st.metric("Average Probability", f"{history_df['prediction'].mean()*100:.1f}%")
    with col3:
        st.metric("Highest Probability", f"{history_df['prediction'].max()*100:.1f}%")
else:
    st.info("No predictions recorded yet. Make your first prediction using the sidebar controls.")

# Footer
st.markdown("---")
st.markdown("**Note:** All predictions are stored in MySQL database for future reference.")

: 